In [ ]:
''' 
This python script is for SPICE uncertainty analysis
aimed for ESMRMB 2025
'''
%load_ext autoreload
%autoreload 2
# import libs
import matplotlib.pyplot as plt
import numpy as np
from fsl_mrs.utils import synthetic as syn
from fsl_mrs.core import basis
from fsl_mrs.utils.plotting import FID2Spec
from fsl_mrs.utils.misc import FIDToSpec
from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg import cg
from scipy.optimize import minimize
from fsl_mrs.utils import mrs_io 
from datetime import datetime
import seaborn as sns
from SPICE_Uncert_Ancillary import (
    createSpatialCurve,
    createSpatialCurve_1D_Brain,
    make_1d_spectral_phantom,
    calc_F,
    gen_gt_ktspace,
    make_synthetic_fid,
    add_noise2kt,
    plot_and_rmse,
    read_training_data_from_csv,
    save_training_data_as_csv,
    CreateWaterImage,
    constraints_to_B,
    Plot_W_WE,
    SPICEWithSpatialConstrain,
    Undersampe_zeroout,
    read_basis_json,
    gen_voxel_signal,


    plot_spec_ana,
    plot_mc_compare_spec,
    plot_mc_compare_spat,
    runmc,
    calc_Covariance_spat,
    calc_std_uncert,
    plot_spec_mc,
    plot_spec_analyt,
    plot_spatial_mc,
    plot_spatial_mc_ana_combined,
    calc_Covariance_spat_overall,
    calc_Covariance_old,
    complex_multivariate_normal,
    Create_laplacian_samples,

    plot_bm_and_bmFID,
    fit_mrs_spectrum_nonlinear,
    fit_mrs_spectrum_nonlinear_ls,
    fit_mrs_spectrum_nonlinear_vbv,
    fit_mrs_spectrum_lstsq_batch_vbv,
    plot_popt,
    Sig_func_Multi_Peak,
    mc_basis,
    mc_basis_vbv
)

In [ ]:
'''
Make Macro Definitions
'''
NUM_SPICE_RANK = 5
UNDER_SAMPLE_NUM = 16
UNDER_SAMPLE_HDL = True
UNDERSAMPLE_RATIO = 25


Peak_CS = [-3.05, -1.5] # basis concentrations
Peak_lws_gt = [10, 10]
Peak_lws_gt_mc = [10, 10]
NUM_SPICE_BASIS = len(Peak_CS)
K_POINTS = 64
N_VOXEL = 64
N_SEQ_POINTS = 512#1024#2048#8192 #512
N_SEQ_BANDWIDTH = 2800#3050
LAMBDA_WE_max = 500#500
SUBSPACE_DATA_RW = True
SAVE_DIR='./Train_data_SPICE/'
CSV_FILE_NAME = 'SS250506'#'SS250430'#'SS250111'
BASIS_DIR='./Basis_Fit_ESMRMB2025/'
BASIS_Cho = 'Cho'
BASIS_Cr = 'Cr'
BASIS_NAA = 'NAA'
BASIS_Glu = 'Glu'
BRAIN_IMG_DIR = './Brain_img/'
BRAIN_IMG_FILE = 'brain_from_k_space_64.npy'
WATER_PEAK_PPM_CENTER = 4.65
NOISE_SNR = 100
Lamda_1 = 0#0.05
scale_factor = 0.018 # scale down the water image so they can be displayed in same figure
limits = [2e-3,1.25e-2] # define the plot MC canvas limits
PEAK_0_ROUGH_IDX = 381#1560
PEAK_1_ROUGH_IDX = 336#1321

SAVDE_DATA_DIR = './Saved_Data/'

# 获取当前时间，格式为 2025-05-04_15-30-00
timestamp = datetime.now().strftime("%Y-%m-%d_%H")

# 输入运行情况描述
description = f'SNR({NOISE_SNR})_lambda({Lamda_1})_Wmax({LAMBDA_WE_max})_US({UNDERSAMPLE_RATIO})'#input("input[lambda, Wmax, Undersample_Ratio](separate with space): ").strip().replace(" ", "_")


In [ ]:
'''
Describe 1D phantom amplitudes
Plot and construct a 1D Phantom
'''
# phantom_res, phantom_size, x_hr, x_hr_pos, C0_GT, C1_GT, combined_amp = createSpatialCurve(K_POINTS)


brain_img = np.load(BRAIN_IMG_DIR+BRAIN_IMG_FILE)
print(brain_img.shape)
print(brain_img.dtype)
plt.imshow(np.abs(brain_img), cmap='viridis')
plt.title('Brain Image from K-space')
plt.axis('on')
plt.show()

SLICE_1D = brain_img[:,25] #12
plt.plot(np.abs(SLICE_1D))
plt.title("1D Slice of Brain Image (Magnitude)")
plt.xlabel("Row Index")
plt.ylabel("Intensity")
plt.grid(True)
plt.show()

SLICE_SMOOTHED = np.round(SLICE_1D)  # 四舍五入为整数
plt.plot(SLICE_SMOOTHED)
plt.title("Rounded/Quantized Slice")
plt.grid(True)
plt.show()


phantom_res, phantom_size, x_hr, x_hr_pos, C0_GT, C1_GT = createSpatialCurve_1D_Brain(K_POINTS,SLICE_SMOOTHED)

# _, _, TIME_AXIS, PPM_AXIS = make_synthetic_fid(Peak_CS[0],10)
# basis_test0_json = read_basis_json(BASIS_DIR, BASIS_NAA)
# basis_test0_FID_re = np.array(basis_test0_json['basis']['basis_re'])
# basis_test0_FID_im = np.array(basis_test0_json['basis']['basis_im'])


fullbasis = mrs_io.read_basis(BASIS_DIR) 
basis = fullbasis.get_formatted_basis(bandwidth=N_SEQ_BANDWIDTH, points=N_SEQ_POINTS)#ignore=[list of metabolites you don't want to include] 
#shape(points,basis_num)
Basis_NAA_FID = basis[:,3] #complex128
Basis_Cho_FID = basis[:,0]
Basis_Cr_FID  = basis[:,1]
Basis_Glu_FID = basis[:,2]



# basis_test0_FID = basis_test0_FID_re + 1j * basis_test0_FID_im
# basis_test0_SPEC = FID2Spec(basis_test0_FID)

sweepwidth = N_SEQ_BANDWIDTH#fullbasis.original_bw #N_SEQ_BANDWIDTH  #basis_test0_json['seq']['Rx_SW']
dwelltime = fullbasis.original_dwell #1/N_SEQ_BANDWIDTH   #basis_test0_json['basis']['basis_dwell']
center_freq = fullbasis._cf
original_points = fullbasis.original_points
# PPM_AXIS = fullbasis.original_ppm_shift_axis

FREQ_AXIS = np.linspace(-sweepwidth / 2, sweepwidth / 2, N_SEQ_POINTS)#np.arange(-sweepwidth/2, sweepwidth/2, sweepwidth/N_SEQ_POINTS)
PPM_AXIS = -(FREQ_AXIS/center_freq) + WATER_PEAK_PPM_CENTER
TIME_AXIS = np.linspace(0.0, dwelltime * (original_points - 1), N_SEQ_POINTS)



#Read and Create the Basis FIDs
# Basis0_FID_json = read_basis_json(BASIS_DIR, BASIS_NAA)
# print('Basis0:', Basis0_FID_json['basis']['basis_name'])
# Basis0_FID_re = np.array(Basis0_FID_json['basis']['basis_re'])
# Basis0_FID_im = np.array(Basis0_FID_json['basis']['basis_im'])
# Basis0_FID = Basis0_FID_re + 1j * Basis0_FID_im
Basis0_FID = Basis_Glu_FID
# Basis0_FID = gen_voxel_signal([10],[1],[0],[Basis0_FID],TIME_AXIS)
Basis0_FID_SPEC = FID2Spec(Basis0_FID)

# print(Basis0_FID_json['basis'])
# print(Basis0_FID_json['seq']['Rx_Points'])
# print(Basis0_FID_json['seq']['Rx_SW'])
# print(len(Basis0_FID_json['basis']['basis_re']))
# print(Basis0_FID_json['basis']['basis_centre'])
# print(Basis0_FID_json['basis']['basis_width'])
# print(Basis0_FID_json['basis']['basis_dwell'])

plt.plot(PPM_AXIS, Basis0_FID_SPEC, label=f'Metabolite Glu')
plt.xlabel('PPM axis')
plt.ylabel('Signal Spectrum')
# plt.xlim([PPM_AXIS.max(), PPM_AXIS.min()])
plt.title('Basis Glu Spectrum')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

plt.plot(TIME_AXIS, np.real(Basis0_FID), label=f'Metabolite Glu')
plt.xlabel('Time axis')
plt.ylabel('Signal')
plt.title('Basis Glu FID(real part)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


# Basis 1
# Basis1_FID_json = read_basis_json(BASIS_DIR, BASIS_Cho)
# print('Basis1:', Basis1_FID_json['basis']['basis_name'])
# Basis1_FID_re = np.array(Basis1_FID_json['basis']['basis_re'])
# Basis1_FID_im = np.array(Basis1_FID_json['basis']['basis_im'])
# Basis1_FID = Basis1_FID_re + 1j * Basis1_FID_im
Basis1_FID = Basis_Cho_FID
Basis1_FID_SPEC = FID2Spec(Basis1_FID)

# print(Basis1_FID_json['basis'])
# print(Basis1_FID_json['seq']['Rx_Points'])
# print(Basis1_FID_json['seq']['Rx_SW'])
# print(len(Basis1_FID_json['basis']['basis_re']))
# print(Basis1_FID_json['basis']['basis_centre'])
# print(Basis1_FID_json['basis']['basis_width'])
# print(Basis1_FID_json['basis']['basis_dwell'])

plt.plot(PPM_AXIS, Basis1_FID_SPEC, label=f'Metabolite Cho')
plt.xlabel('PPM axis')
plt.ylabel('Signal Spectrum')
# plt.xlim([PPM_AXIS.max(), PPM_AXIS.min()])
plt.title('Basis Cho Spectrum')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

plt.plot(TIME_AXIS, np.real(Basis1_FID), label=f'Metabolite Cho')
plt.xlabel('Time axis')
plt.ylabel('Signal')
plt.title('Basis Cho FID(real part)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

bm_FIDs = [Basis0_FID,Basis1_FID]

# assert np.allclose(Basis_NAA_FID, Basis0_FID, atol=1e-6), "Basis_NAA_FID and Basis0_FID are not close enough!"
# assert np.allclose(Basis_Cho_FID, Basis1_FID, atol=1e-6), "Basis_Cho_FID and Basis1_FID are not close enough!"


''' 
Single Peak Basis for Testing only
'''
# Basis0_FID_old,_,_,_ = make_synthetic_fid(Peak_CS[0],0) #Single Peak Basis for Testing only
# Basis1_FID_old,_,_,_ = make_synthetic_fid(Peak_CS[1],0) #Single Peak Basis for Testing only
# bm_FIDs = [Basis0_FID,Basis1_FID] #Single Peak Basis for Testing only

# #Assert to make sure the old FID and real FID have the same data structure and type
# assert type(Basis0_FID_old) == type(Basis0_FID), f"Type mismatch: {type(Basis0_FID_old)} vs {type(Basis0_FID)}"
# assert Basis0_FID_old.shape == Basis0_FID.shape, f"Shape mismatch: {Basis0_FID_old.shape} vs {Basis0_FID.shape}"
# assert Basis0_FID_old.dtype == Basis0_FID.dtype, f"Dtype mismatch: {Basis0_FID_old.dtype} vs {Basis0_FID.dtype}"


# print(Basis0_FID_old[:5])  # 打印前5个元素
# print(Basis0_FID[:5])
''' 
Single Peak Basis for Testing only
'''


GT_IT_SPACE = make_1d_spectral_phantom(C0_GT,C1_GT,Peak_CS,Peak_lws_gt,bm_FIDs,TIME_AXIS) #FID*Image spaceX

F = calc_F(K_POINTS,True)
GT_KT_SPACE = gen_gt_ktspace(GT_IT_SPACE,K_POINTS,F,TIME_AXIS,True)

rng = np.random.default_rng()
NOISE_SD = np.max(np.abs(GT_KT_SPACE)) / NOISE_SNR
noisy_kt_space = add_noise2kt(GT_KT_SPACE, rng, NOISE_SD)

# Undersample
noisy_kt_space_us_eg,F_us_eg = Undersampe_zeroout(noisy_kt_space,F,UNDER_SAMPLE_NUM,K_POINTS,TIME_AXIS,True, UNDER_SAMPLE_HDL)


k_x = np.arange(K_POINTS)
plt.pcolor(TIME_AXIS, k_x, np.abs(noisy_kt_space))
plt.title('noisy kt-space, truncated view in t')
plt.xlabel('Time (#)')
plt.xlim([0, TIME_AXIS[50]])
plt.ylabel('$k_x$')
plt.show()





In [ ]:
cmax_training = 1.5
lw_sd_training = 2
lw_mean_training = 10
training_datasets = int(1E4)


# tissue 1, chem 1 is 2x chem 2 concentration
training_cs = rng.random((2, training_datasets)) * cmax_training

training_lw = lw_mean_training + rng.standard_normal((2, training_datasets)) * lw_sd_training

training_dataset = Sig_func_Multi_Peak(bm_FIDs,training_lw,training_cs,TIME_AXIS,training_datasets)


plt.plot(PPM_AXIS, FID2Spec(training_dataset[:5, :].T).real)
plt.show()


# save the training date to csv
save_training_data_as_csv(training_data=training_dataset,save_dir=SAVE_DIR,filename=CSV_FILE_NAME,savecondition=SUBSPACE_DATA_RW)

In [ ]:
'''
Take the svd of the data
'''
# Read the training data from the csv
training_dataset = read_training_data_from_csv(save_dir=SAVE_DIR,filename=CSV_FILE_NAME)


u,s,vh = np.linalg.svd(training_dataset)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
ax1.plot(s, 'x--')
ax1.set_xlim([0, 20])
# Set the x-axis to display only integer gridlines
ax1.xaxis.set_major_locator(plt.MaxNLocator(integer=True))

ax2.plot(PPM_AXIS, FID2Spec(vh[0:NUM_SPICE_RANK,:].T).real)
# ax2.set_xlim([0, PPM_AXIS[0]])
plt.show()



In [ ]:
#1. create water image
# water_rou_1D = CreateWaterImage(K_POINTS)
# Replace all elements equal to 1 with 0
# Replace all 1s with 0
SLICE_SMOOTHED[SLICE_SMOOTHED == 1] = 0
SLICE_SMOOTHED[SLICE_SMOOTHED == 2] = 3

# Make a copy if needed
SLICE_SMOOTHED_extracted = SLICE_SMOOTHED.copy()

# Plot as a line
plt.figure(figsize=(8, 4))
plt.plot(SLICE_SMOOTHED_extracted, marker='o')
plt.title("SLICE_SMOOTHED with 1s set to 0")
plt.xlabel("Index")
plt.ylabel("Value")
plt.grid(True)
plt.tight_layout()
plt.show()

water_rou_1D = SLICE_SMOOTHED_extracted

#2. Calculate Edge Preserving Matrix
# calculate the edge preserving matrix W_edge for given constraint
minpooling_Handler = True # use min pooling
pool_size = 1 # can adjust the pool size here
W_max = LAMBDA_WE_max
lamda_1 = Lamda_1
W_edge, _W, _P = constraints_to_B(water_rou_1D, W_max=W_max, pool_size=pool_size) 
Plot_W_WE(_W, _P)


V = vh[0:NUM_SPICE_RANK, :].conj().T
Vh = vh[0:NUM_SPICE_RANK, :]

if UNDER_SAMPLE_HDL:
    spice_est_cg,est_U = SPICEWithSpatialConstrain(noisy_kt_space_us_eg,F_us_eg,V,K_POINTS,NUM_SPICE_RANK,W_edge,'cg',lamda_1)
else:
    spice_est_cg,est_U = SPICEWithSpatialConstrain(noisy_kt_space,F,V,K_POINTS,NUM_SPICE_RANK,W_edge,'cg',lamda_1)

GT_spice_est_cg, GT_est_U = SPICEWithSpatialConstrain(GT_KT_SPACE,F,V,K_POINTS,NUM_SPICE_RANK,W_edge,'cg',0)

''' 
Code for search the main peak index for demon stration, 
change the plot_and_rmse()->plot_feq_bin()'s idx accordingly
'''
# 如果是复数谱图，取模长（幅值）最大点
main_peak_index = np.argmax(np.abs(Basis0_FID_SPEC))

print(f"Basis0 Main peak index: {main_peak_index}")
main_peak_ppm = PPM_AXIS[main_peak_index]
print(f"Basis0 Main peak PPM: {main_peak_ppm}")

# 如果是复数谱图，取模长（幅值）最大点
main_peak_index = np.argmax(np.abs(Basis1_FID_SPEC))
print(f"Basis1 Main peak index: {main_peak_index}")
main_peak_ppm = PPM_AXIS[main_peak_index]
print(f"Basis1 Main peak PPM: {main_peak_ppm}")

''' 
Code for search the main peak index for demon stration, 
change the plot_and_rmse()->plot_feq_bin()'s idx accordingly
'''


plot_and_rmse(spice_est_cg,GT_IT_SPACE,PPM_AXIS,x_hr_pos,C0_GT,C1_GT)

In [ ]:
'''
MC Method for cg
'''
spice_mc_cg,_ = runmc(SPICEWithSpatialConstrain,add_noise=add_noise2kt,gen_undersample=Undersampe_zeroout, 
                       UNDER_SAMPLE_NUM=UNDER_SAMPLE_NUM, kt_space_gt=GT_KT_SPACE,F=F, K_POINTS=K_POINTS, time_axis=TIME_AXIS, handler=UNDER_SAMPLE_HDL,
                       W_edge=W_edge, lamda_1=Lamda_1,Solver='cg', iterations=2000,seed_mc=42,noise_SD =NOISE_SD, V= V, NUM_SPICE_RANK= NUM_SPICE_RANK)

# SPICE.plot_mc_results(spice_mc_cg, plot_and_rmse=PLF.plot_and_rmse, ppm_axis= ppm_axis, water_rou_1D = water_rou_1D)

'''
MC Method for analytical
'''
spice_mc_analy,_ = runmc(SPICEWithSpatialConstrain,add_noise=add_noise2kt,gen_undersample=Undersampe_zeroout, 
                       UNDER_SAMPLE_NUM=UNDER_SAMPLE_NUM, kt_space_gt=GT_KT_SPACE,F=F, K_POINTS=K_POINTS, time_axis=TIME_AXIS, handler=UNDER_SAMPLE_HDL,
                       W_edge=W_edge, lamda_1=Lamda_1,Solver='analytical', iterations=2000,seed_mc=42,noise_SD =NOISE_SD, V= V, NUM_SPICE_RANK= NUM_SPICE_RANK)

# SPICE.plot_mc_results(spice_mc_analy, plot_and_rmse=PLF.plot_and_rmse, ppm_axis= ppm_axis, water_rou_1D = water_rou_1D)

# save data
filename = f"spice_mc_cg_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, spice_mc_cg)
print(f"数据已保存为文件：{filename}")

filename = f"spice_mc_analy_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, spice_mc_analy)
print(f"数据已保存为文件：{filename}")
pass


In [ ]:
'''
Analytical Method for Uncertainty
'''
uncert_array = []

# Iterate over freq from 0 to 511.
for freq in np.arange(N_SEQ_POINTS):
    cov_1 = calc_Covariance_spat(NOISE_SD, Lamda_1, W_edge, K_POINTS,V, freq)
    uncert_1 = calc_std_uncert(cov_1)
    uncert_array.append(np.array(uncert_1))


uncert_array = np.vstack(uncert_array) # turn into 2D numpy array
print('shape of uncert_array:', uncert_array.shape)
plt.matshow(abs(uncert_array))

# Calculate the variance of each row
variances = np.var(uncert_array, axis=1)

# # Find the row number with the highest variance
# max_variance_index = np.argmax(variances)
max_variance_index = np.argpartition(variances, -5)[-5:]
max_variance_index = max_variance_index[np.argsort(variances[max_variance_index])[::-1]]


print(f"max_variance_index: {max_variance_index}")


cov_p1 = calc_Covariance_spat(NOISE_SD, Lamda_1, W_edge,K_POINTS,V, PEAK_0_ROUGH_IDX)#64
uncert_p1 = calc_std_uncert(cov_p1)

cov_p2 = calc_Covariance_spat(NOISE_SD, Lamda_1, W_edge,K_POINTS,V, PEAK_1_ROUGH_IDX)#161
uncert_p2 = calc_std_uncert(cov_p2)



uncert_array = []

# Iterate over freq from 0 to 511.
for freq in np.arange(N_SEQ_POINTS):
    cov_1 = calc_Covariance_spat(NOISE_SD, Lamda_1, W_edge,K_POINTS,V,freq)
    uncert_1 = calc_std_uncert(cov_1)
    uncert_array.append(np.array(uncert_1))


uncert_array = np.vstack(uncert_array) # turn into 2D numpy array

'''Plot out the results'''
plot_mc_compare_spec(spice_mc_cg,spice_mc_analy,res_array3=uncert_array, plot_spec_mc = plot_spec_mc,plot_spec_analyt=plot_spec_analyt,ppm_axis=PPM_AXIS,limits =limits) #[2, 8, 16, 24, 30]
plot_mc_compare_spat(spice_mc_cg,spice_mc_analy,plot_spatial_mc=plot_spatial_mc,water_rou_1D=water_rou_1D,x_hr_pos = x_hr_pos, limits=limits)
mc_rec, laplac = plot_spatial_mc_ana_combined(mc_cg=spice_mc_cg,mc_analyt=spice_mc_analy,std_uncert1_analyt=uncert_p1,std_uncert2_analyt=uncert_p2,water_rou_1D=water_rou_1D,x_hr_pos=x_hr_pos)


In [ ]:
''' 
MRSI fit
'''





# Call the function
plot_bm_and_bmFID(bm_FIDs,PPM_AXIS)


Cm_general = [C0_GT,C1_GT]
y = np.real(Sig_func_Multi_Peak(bm_FIDs,Peak_lws_gt,Cm_general,TIME_AXIS,N_VOXEL))
y_p = np.real(GT_IT_SPACE)

assert np.allclose(y, y_p, atol=1e-6), "y and GT_IT_SPACE are not close enough!"



spectrum = spice_est_cg#GT_IT_SPACE#spice_est_cg#est_U @ Vh
# spectrum = spectrum.flatten()  # Flatten to 1D array (16384,)

# Flatten initial guesses into a single 1D array
initial_guesses = [8,2,8,2]#list(np.ones(N_VOXEL)*10) + list(np.ones(N_VOXEL)*0.5) + list(np.ones(N_VOXEL)*10) + list(np.ones(N_VOXEL)*0.5)  
initial_guesses = np.array(initial_guesses, dtype=np.float64)


    
    

# popt, pcov = fit_mrs_spectrum_nonlinear_ls(bm_FIDs, PPM_AXIS, spectrum, initial_guesses)
popt, pcov = fit_mrs_spectrum_lstsq_batch_vbv(bm_FIDs,TIME_AXIS,spectrum,initial_guesses)#fit_mrs_spectrum_nonlinear(bm_FIDs,TIME_AXIS,spectrum,initial_guesses)
print("LW1 parameters:", popt[0:N_VOXEL])
print("LW2 parameters:", popt[2*N_VOXEL:3*N_VOXEL])



lw1 = popt[0:N_VOXEL]
fit1 = popt[N_VOXEL:2*N_VOXEL]
lw2 = popt[2*N_VOXEL:3*N_VOXEL]
fit2 = popt[3*N_VOXEL:4*N_VOXEL]
plot_popt(popt)






In [ ]:
# def fft_recon(noisy_kt_spaces):
#     fft_recon = np.fft.fftshift(
#         np.fft.fft(
#                 np.fft.fftshift(noisy_kt_spaces, axes=0),
#                 axis=0, norm = 'ortho'), #norm='forward'
#             axes=0)
#     return fft_recon[::-1]

def fft_recon(noisy_kt_spaces):
    out = np.fft.ifftshift(
        np.fft.fft(
            np.fft.fftshift(noisy_kt_spaces, axes=0),
            axis=0, norm='ortho'),
        axes=0)
    return np.roll(out[::-1], shift=1, axis=0)



def fft_mc(recon_func:callable, 
          add_noise:callable,
          gen_undersample:callable,
          UNDER_SAMPLE_NUM: int,  
          seed_mc:int,
          kt_space_gt:np.ndarray,
          F:np.ndarray,
          K_POINTS:int, 
          time_axis:np.ndarray, 
          noise_SD:float,
          iterations:int=500,
          handler:bool = False,) -> tuple:
    output = []
    rng = np.random.default_rng(seed=seed_mc)  # create rng
    # kt_groundtruth = np.copy(self.kt_space_gt)
    # print("I've reloaded")
    for _ in range(iterations):
        # kt_groundtruth = self.kt_space_gt
        gen_ktspace = add_noise(kspace=kt_space_gt, rng=rng, noise_SD = noise_SD)
        # assert np.allclose(self.kt_space_gt, kt_groundtruth)
        gen_undersample_result, new_F = gen_undersample(noisy_kt_space_US = gen_ktspace,F_US = F,  
                                                    UNDER_SAMPLE_NUM = UNDER_SAMPLE_NUM, K_POINTS= K_POINTS, time_axis = time_axis, handler = handler)
        result_spec_est = recon_func(noisy_kt_spaces = gen_undersample_result)
        output.append(result_spec_est)
        # Print current iteration number every 50 steps
        if _ % 50 == 0:
            print(f"Iteration {_} completed.")
    return np.asarray(output)

In [ ]:
if UNDER_SAMPLE_HDL:
    spice_est_fft = fft_recon(noisy_kt_space_us_eg) 
else:
    spice_est_fft = fft_recon(noisy_kt_space)#GT_KT_SPACE


spectrum = spice_est_fft#GT_IT_SPACE#spice_est_cg#est_U @ Vh
# spectrum = spectrum.flatten()  # Flatten to 1D array (16384,)

# Flatten initial guesses into a single 1D array
initial_guesses = [8,2,8,2] 
initial_guesses = np.array(initial_guesses, dtype=np.float64)

# popt, pcov = fit_mrs_spectrum_nonlinear_ls(bm_FIDs, PPM_AXIS, spectrum, initial_guesses)
popt, pcov = fit_mrs_spectrum_lstsq_batch_vbv(bm_FIDs,TIME_AXIS,spectrum,initial_guesses)
print("LW1 parameters:", popt[0:N_VOXEL])
print("LW2 parameters:", popt[2*N_VOXEL:3*N_VOXEL])



lw1 = popt[0:N_VOXEL]
fit1 = popt[N_VOXEL:2*N_VOXEL]
lw2 = popt[2*N_VOXEL:3*N_VOXEL]
fit2 = popt[3*N_VOXEL:4*N_VOXEL]
plot_popt(popt)

In [ ]:
fft_mc_result = fft_mc(fft_recon,add_noise=add_noise2kt,gen_undersample=Undersampe_zeroout, 
                       UNDER_SAMPLE_NUM=UNDER_SAMPLE_NUM, kt_space_gt=GT_KT_SPACE,F=F, K_POINTS=K_POINTS, time_axis=TIME_AXIS, handler=UNDER_SAMPLE_HDL,
                       iterations=2000,seed_mc=42,noise_SD =NOISE_SD)

# shape: (n_iter, N_VOXEL, T)
n_iter, N_VOXEL, T = fft_mc_result.shape

# freq_std_across_voxel[f] = std(fft_mc_result[:, :, f]) across voxels
std_across_voxels = np.std(fft_mc_result, axis=1)  # shape: (n_iter, T)

# 然后计算 mean/std across MC samples → shape: (T,)
mean_std = np.mean(std_across_voxels, axis=0)
std_std = np.std(std_across_voxels, axis=0)

plt.figure(figsize=(10, 5))
plt.plot(mean_std, label="Mean STD across voxels")
plt.fill_between(range(T), mean_std - std_std, mean_std + std_std, alpha=0.3, label="±1 STD band")
plt.title("Per-frequency STD across voxels")
plt.xlabel("Frequency index")
plt.ylabel("STD across voxels")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# spectral
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')

# 计算每个 voxel 的 MC 平均或标准差（你可以改为 mean）
# fft_mc_result: shape (n_iter, N_voxel, T)
uncertainty = np.std(np.real(fft_mc_result), axis=0)  # shape: (N_voxel, T)

voxel_indices = [2, 8, 16, 24, 30]

for voxel in voxel_indices:
    y = np.full_like(PPM_AXIS, voxel)
    z = uncertainty[voxel, :]
    ax.plot(PPM_AXIS, y, z, label=f'Voxel {voxel}')

ax.set_xlim(PPM_AXIS[-1], PPM_AXIS[0])  # 化学位移反向
ax.set_xlabel('$\\delta$ / ppm')
ax.set_ylabel('Voxel #')
ax.set_zlabel('Uncertainty')
ax.view_init(elev=15, azim=-80)
ax.legend()
plt.tight_layout()
plt.show()

# --- Spatial uncertainty at two spectral peaks ---
fig, ax = plt.subplots(figsize=(8, 4))

# 假设 fft_mc_result: (n_iter, N_voxel, T)
peak1est = np.std(np.abs(fft_mc_result[:, :, PEAK_0_ROUGH_IDX]), axis=0)  # (N_voxel,)
peak2est = np.std(np.abs(fft_mc_result[:, :, PEAK_1_ROUGH_IDX]), axis=0)  # (N_voxel,)

xaxis = x_hr_pos  # 横坐标：voxel 位置，或 index
ax.plot(xaxis, peak1est, label='Peak 1 STD')
ax.plot(xaxis, peak2est, label='Peak 2 STD')

ax.set_xlabel('x (voxel position)')
ax.set_ylabel('Uncertainty (STD)')
ax.set_title('Spatial Uncertainty at Two Peaks')
ax.set_ylim([0, max(np.max(peak1est), np.max(peak2est)) * 1.1])
ax.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

filename = f"fft_mc_result_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, fft_mc_result)
print(f"数据已保存为文件：{filename}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from matplotlib.font_manager import FontProperties

bold_font = FontProperties()
bold_font.set_weight('bold')

# ✅ 设置 seaborn 风格
sns.set_theme(style="whitegrid", font_scale=1.2)

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')

# 手动颜色
colors_dft = sns.color_palette("Reds", 15)[-5:]  # 或替换为你自己定义的 RGB
colors_spice = sns.color_palette("Blues", 15, desat=None)[-5:]  # 或替换为你自己定义的 RGB

# colors = ['#f16262', '#d03f35', '#d5306e', '#b849a9', '#9036aa']

# 数据
spice_mc_cg_W_for_plt = np.load(SAVDE_DATA_DIR+"spice_mc_cg_SNR(100)_lambda(0.05)_Wmax(500)_US(0)_2025-05-13_14.npy")
spice_mc_cg_WO_for_plt = np.load(SAVDE_DATA_DIR+"spice_mc_cg_SNR(100)_lambda(0)_Wmax(500)_US(0)_2025-05-13_14.npy")
fft_mc_result_for_plt = np.load(SAVDE_DATA_DIR+"fft_mc_result_SNR(100)_lambda(0)_Wmax(500)_US(0)_2025-05-13_14.npy")

uncertainty = np.std(np.real(fft_mc_result_for_plt), axis=0)
res_array_spec1 = FIDToSpec(spice_mc_cg_W_for_plt, axis=-1).real.std(axis=0)
res_array_spec2 = FIDToSpec(spice_mc_cg_WO_for_plt, axis=-1).real.std(axis=0)
voxel_indices = [4, 16, 28, 40, 52]

# 绘图
 

for idx, voxel in enumerate(voxel_indices):
    x = PPM_AXIS
    # ✅ Clip 到 [0, 5] 区间 —— 避免越界绘图
    mask = (x >= 0) & (x <= 5)
    x_clipped = x[mask]
    y = np.full_like(x_clipped, voxel)
    z_std = uncertainty[voxel, :]
    z_spec_W = res_array_spec1[voxel, :]
    z_spec_WO = res_array_spec2[voxel, :]
    z_std_clipped = z_std[mask]
    z_spec_W_clipped = z_spec_W[mask]
    z_spec_WO_clipped = z_spec_WO[mask]

    ax.plot(x_clipped, y, z_std_clipped, label='FFT' if idx == 0 else None, color=colors_dft[idx], linewidth=2) #color=colors[idx]
    # ax.plot(x_clipped, y, z_spec_W_clipped, label='SPICE_W' if idx == 0 else None, color=colors[idx], alpha=0.6, linewidth=2)
    ax.plot(x_clipped, y, z_spec_WO_clipped, label='SPICE_WO' if idx == 0 else None, color=colors_spice[idx], alpha=0.5, linewidth=2)

# voxels_full = np.arange(len(water_rou_1D))
# x_fixed = np.full_like(water_rou_1D, 3)  # constant 3 ppm
# z_water_full = water_rou_1D * 0.0006       # scale if needed

# ax.plot(x_fixed, voxels_full, z_water_full, color='gray',linestyle=':', linewidth=2, label='Anatomical Reference')

# 轴设定
ax.set_xlim([5, 0])
ax.set_xlabel('$\\delta$  (ppm) ', fontweight='bold')
ax.set_ylabel('Voxel', fontweight='bold')
ax.set_zlabel('Uncertainty', fontweight='bold', labelpad=20)
ax.zaxis.set_label_coords(-0.3, 0.5)

# 视角
ax.view_init(elev=15, azim=-40)

# 图例右侧
# ax.legend(loc='center left', bbox_to_anchor=(1.05, 0.5), fontsize=9, prop=bold_font)

plt.tight_layout()
plt.show()


In [ ]:
bold_font = FontProperties()
bold_font.set_weight('bold')

# ✅ 设置 seaborn 风格
sns.set_theme(style="whitegrid", font_scale=1.2)

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')

# 手动颜色
# colors = sns.color_palette("pastel", 5)  # 或替换为你自己定义的 RGB
colors = ['#f16262', '#d03f35', '#d5306e', '#b849a9', '#9036aa']

# 数据
spice_mc_cg_W_for_plt = np.load(SAVDE_DATA_DIR+"spice_mc_cg_SNR(100)_lambda(0.05)_Wmax(500)_US(0)_2025-05-13_14.npy")
spice_mc_cg_WO_for_plt = np.load(SAVDE_DATA_DIR+"spice_mc_cg_SNR(100)_lambda(0)_Wmax(500)_US(0)_2025-05-13_14.npy")
fft_mc_result_for_plt = np.load(SAVDE_DATA_DIR+"fft_mc_result_SNR(100)_lambda(0)_Wmax(500)_US(0)_2025-05-13_14.npy")

uncertainty = np.std(np.real(fft_mc_result_for_plt), axis=0)
res_array_spec1 = FIDToSpec(spice_mc_cg_W_for_plt, axis=-1).real.std(axis=0)
res_array_spec2 = FIDToSpec(spice_mc_cg_WO_for_plt, axis=-1).real.std(axis=0)
voxel_indices = [4, 16, 28, 40, 52]

# 绘图
 

for idx, voxel in enumerate(voxel_indices):
    x = PPM_AXIS
    # ✅ Clip 到 [0, 5] 区间 —— 避免越界绘图
    mask = (x >= 0) & (x <= 5)
    x_clipped = x[mask]
    y = np.full_like(x_clipped, voxel)
    z_std = uncertainty[voxel, :]
    z_spec_W = res_array_spec1[voxel, :]
    z_spec_WO = res_array_spec2[voxel, :]
    z_std_clipped = z_std[mask]
    z_spec_W_clipped = z_spec_W[mask]
    z_spec_WO_clipped = z_spec_WO[mask]

    # ax.plot(x_clipped, y, z_std_clipped, label='FFT' if idx == 0 else None, color=colors[idx], linewidth=2) #color=colors[idx]
    ax.plot(x_clipped, y, z_spec_WO_clipped, label='SPICE_WO' if idx == 0 else None,linestyle='--', color=colors_spice[idx], alpha=0.5, linewidth=2)
    ax.plot(x_clipped, y, z_spec_W_clipped, label='SPICE_W' if idx == 0 else None, color=colors_spice[idx], alpha=1, linewidth=2.5)

voxels_full = np.arange(len(water_rou_1D))
x_fixed = np.full_like(water_rou_1D, 3)  # constant 3 ppm
z_water_full = water_rou_1D * 0.0006       # scale if needed

ax.plot(x_fixed, voxels_full, z_water_full, color='gray', linewidth=2, label='Anatomical Reference')

# 轴设定
ax.set_xlim([5, 0])
ax.set_xlabel('$\\delta$  (ppm) ', fontweight='bold')
ax.set_ylabel('Voxel', fontweight='bold')
ax.set_zlabel('Uncertainty', fontweight='bold', labelpad=20)
ax.zaxis.set_label_coords(-0.3, 0.5)

# 视角
ax.view_init(elev=15, azim=-40)

# 图例右侧
# ax.legend(loc='center left', bbox_to_anchor=(1.05, 0.5), fontsize=9, prop=bold_font)

plt.tight_layout()
plt.show()


In [ ]:
assert np.all(np.isfinite(spectrum)), "Spectrum contains NaN or Inf"
assert np.all(np.isfinite(bm_FIDs)), "bm_FIDs contains NaN or Inf"
print("Check spectrum:", np.all(np.isfinite(spectrum)))
print("Check time_axis:", np.all(np.isfinite(TIME_AXIS)))
print("Check bm_FIDs:", np.all(np.isfinite(bm_FIDs)))
print("Check initial_guesses:", np.all(np.isfinite(initial_guesses)))

cm1_fft,cm2_fft,lw1_fft,lw2_fft = mc_basis(fft_mc_result, bm_FIDs, TIME_AXIS)

cm1_mean = np.mean(cm1_fft,axis=0)
cm2_mean = np.mean(cm2_fft,axis=0)
lw1_mean = np.mean(lw1_fft,axis=0)
lw2_mean = np.mean(lw2_fft,axis=0)

# Plot results with error bars
plt.figure(figsize=(8, 5))
plt.plot(cm1_mean, marker='o', linestyle='-',  label="Cm1 mean")
plt.plot(cm2_mean, marker='s', linestyle='-',  label="Cm2 mean")
plt.plot( water_rou_1D*0.02, label='Water ROU 1D', linestyle=':')

plt.xlabel("Voxel Index")
plt.ylabel("Fitted Cm Value")
plt.title("MC-Based MRS Fitting Results average")
plt.legend()
plt.show()

cm1_fft_std = np.std(cm1_fft, axis=0)
cm2_fft_std = np.std(cm2_fft, axis=0)

# Plot results with error bars
plt.figure(figsize=(8, 5))
plt.plot(cm1_fft_std, marker='o', linestyle='-',  label="Cm1 spatial uncert")
plt.plot(cm2_fft_std, marker='s', linestyle='-',  label="Cm2 spatial uncert")
plt.plot( water_rou_1D*0.02, label='Water ROU 1D', linestyle=':')

plt.xlabel("Voxel Index")
plt.ylabel("Fitted Cm Value")
plt.title("MC-Based MRS Fitting Results with Uncertainty")
plt.legend()
plt.show()

lw1_fft_std = np.std(lw1_fft, axis=0)
lw2_fft_std = np.std(lw2_fft, axis=0)

# Plot results with error bars
plt.figure(figsize=(8, 5))
plt.plot(lw1_fft_std, marker='o', linestyle='-',  label="Cm1 spatial uncert")
plt.plot(lw2_fft_std, marker='s', linestyle='-',  label="Cm2 spatial uncert")
plt.plot( water_rou_1D*2, label='Water ROU 1D', linestyle=':')

plt.xlabel("Voxel Index")
plt.ylabel("Fitted lw Value")
plt.title("MC-Based MRS Lw Fitting Results with Uncertainty")
plt.legend()
plt.show()

# save data
filename = f"cm1_fft_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, cm1_fft)
print(f"数据已保存为文件：{filename}")

filename = f"cm2_fft_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, cm2_fft)
print(f"数据已保存为文件：{filename}")

filename = f"lw1_fft_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, lw1_fft)
print(f"数据已保存为文件：{filename}")

filename = f"lw2_fft_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, lw2_fft)
print(f"数据已保存为文件：{filename}")

filename = f"cm1_fft_std_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, cm1_fft_std)
print(f"数据已保存为文件：{filename}")

filename = f"cm2_fft_std_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, cm2_fft_std)
print(f"数据已保存为文件：{filename}")

filename = f"lw1_fft_std_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, lw1_fft_std)
print(f"数据已保存为文件：{filename}")

filename = f"lw2_fft_std_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, lw2_fft_std)
print(f"数据已保存为文件：{filename}")

In [ ]:
# spice_est_cg_set = np.stack([spice_est_cg, spice_est_cg], axis=0)
# print(spice_mc_cg.shape)
# cm1,cm2,lw1,lw2 = mc_basis(spice_est_cg_set, bm_FIDs, TIME_AXIS)


cm1,cm2,lw1,lw2 = mc_basis(spice_mc_cg, bm_FIDs, TIME_AXIS)

cm1_mean = np.mean(cm1,axis=0)
cm2_mean = np.mean(cm2,axis=0)
lw1_mean = np.mean(lw1,axis=0)
lw2_mean = np.mean(lw2,axis=0)

# Plot results with error bars
plt.figure(figsize=(8, 5))
plt.plot(cm1_mean, marker='o', linestyle='-',  label="Cm1 mean")
plt.plot(cm2_mean, marker='s', linestyle='-',  label="Cm2 mean")
plt.plot( water_rou_1D*0.02, label='Water ROU 1D', linestyle=':')

plt.xlabel("Voxel Index")
plt.ylabel("Fitted Cm Value")
plt.title("MC-Based MRS Fitting Results average")
plt.legend()
plt.show()

cm1_std = np.std(cm1, axis=0)
cm2_std = np.std(cm2, axis=0)

# Plot results with error bars
plt.figure(figsize=(8, 5))
plt.plot(cm1_std, marker='o', linestyle='-',  label="Cm1 spatial uncert")
plt.plot(cm2_std, marker='s', linestyle='-',  label="Cm2 spatial uncert")
plt.plot( water_rou_1D*0.02, label='Water ROU 1D', linestyle=':')

plt.xlabel("Voxel Index")
plt.ylabel("Fitted Cm Value")
plt.title("MC-Based MRS Fitting Results with Uncertainty")
plt.legend()
plt.show()

lw1_std = np.std(lw1, axis=0)
lw2_std = np.std(lw2, axis=0)

# Plot results with error bars
plt.figure(figsize=(8, 5))
plt.plot(lw1_std, marker='o', linestyle='-',  label="Cm1 spatial uncert")
plt.plot(lw2_std, marker='s', linestyle='-',  label="Cm2 spatial uncert")
plt.plot( water_rou_1D*0.02, label='Water ROU 1D', linestyle=':')

plt.xlabel("Voxel Index")
plt.ylabel("Fitted lw Value")
plt.title("MC-Based MRS Lw Fitting Results with Uncertainty")
plt.legend()
plt.show()

# save data
filename = f"cm1_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, cm1)
print(f"数据已保存为文件：{filename}")

filename = f"cm2_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, cm2)
print(f"数据已保存为文件：{filename}")

filename = f"lw1_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, lw1)
print(f"数据已保存为文件：{filename}")

filename = f"lw2_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, lw2)
print(f"数据已保存为文件：{filename}")

filename = f"cm1_std_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, cm1_std)
print(f"数据已保存为文件：{filename}")

filename = f"cm2_std_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, cm2_std)
print(f"数据已保存为文件：{filename}")

filename = f"lw1_std_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, lw1_std)
print(f"数据已保存为文件：{filename}")

filename = f"lw2_std_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, lw2_std)
print(f"数据已保存为文件：{filename}")

In [ ]:
''' 
Laplacian MC for fitting uncertainty
'''
cov_overall = calc_Covariance_spat_overall(NOISE_SD, Lamda_1, W_edge,K_POINTS,V)

cov_overall_old = calc_Covariance_old(NOISE_SD, Lamda_1, W_edge,K_POINTS)

assert np.allclose(cov_overall, cov_overall_old, atol=1e-6), "cov_overall and cov_overall_old are not close enough!"

# Laplac_data = Create_laplacian_samples(GT_est_U,Vh, cov_overall,2000)
Laplac_data = Create_laplacian_samples(est_U,Vh, cov_overall,2000)

print(Laplac_data.shape)
print(spice_mc_cg.shape)
# save data
filename = f"cov_overall_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, cov_overall)
print(f"数据已保存为文件：{filename}")

filename = f"Laplac_data_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, Laplac_data)
print(f"数据已保存为文件：{filename}")

result = np.diag((W_edge.conj().T @ W_edge).toarray())
plt.figure(figsize=(8, 5))
plt.plot(result)

plt.plot( water_rou_1D*2, label='Water ROU 1D', linestyle=':')

plt.legend()
plt.show()
# print('result',result.shape)

# print("Type:", type(result))
# print("Shape:", getattr(result, 'shape', 'No shape attribute'))

# # Plot results with error bars
# plt.matshow(result.toarray())
# # plt.plot( water_rou_1D*0.02, label='Water ROU 1D', linestyle=':')

# plt.legend()
# plt.show()



cm1_laplac,cm2_laplac,lw1_laplac,lw2_laplac = mc_basis(Laplac_data, bm_FIDs, TIME_AXIS)


cm1_mean = np.mean(cm1_laplac,axis=0)
cm2_mean = np.mean(cm2_laplac,axis=0)
lw1_mean = np.mean(lw1_laplac,axis=0)
lw2_mean = np.mean(lw2_laplac,axis=0)

# Plot results with error bars
plt.figure(figsize=(8, 5))
plt.plot(cm1_mean, marker='o', linestyle='-',  label="Cm1 mean")
plt.plot(cm2_mean, marker='s', linestyle='-',  label="Cm2 mean")
plt.plot( water_rou_1D*0.02, label='Water ROU 1D', linestyle=':')

plt.xlabel("Voxel Index")
plt.ylabel("Fitted Cm Value")
plt.title("MC-Based MRS Fitting Results average")
plt.legend()
plt.show()


cm1_laplac_std = np.std(cm1_laplac, axis=0)
cm2_laplac_std = np.std(cm2_laplac, axis=0)

# Plot results with error bars
plt.figure(figsize=(8, 5))
plt.plot(cm1_laplac_std, marker='o', linestyle='-',  label="Cm1 spatial uncert")
plt.plot(cm2_laplac_std, marker='s', linestyle='-',  label="Cm2 spatial uncert")
plt.plot( water_rou_1D*0.02, label='Water ROU 1D', linestyle=':')

plt.xlabel("Voxel Index")
plt.ylabel("Fitted Cm Value")
plt.title("Laplacian-approx MC-Based MRS Fitting Results with Uncertainty")
plt.legend()
plt.show()

lw1_laplac_std = np.std(lw1_laplac, axis=0)
lw2_laplac_std = np.std(lw2_laplac, axis=0)

# Plot results with error bars
plt.figure(figsize=(8, 5))
plt.plot(lw1_laplac_std, marker='o', linestyle='-',  label="Cm1 spatial uncert")
plt.plot(lw2_laplac_std, marker='s', linestyle='-',  label="Cm2 spatial uncert")
plt.plot( water_rou_1D*0.02, label='Water ROU 1D', linestyle=':')

plt.xlabel("Voxel Index")
plt.ylabel("Fitted lw Value")
plt.title("Laplacian-approx MC-Based MRS Lw Fitting Results with Uncertainty")
plt.legend()
plt.show()

# save data
filename = f"cm1_laplac_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, cm1_laplac)
print(f"数据已保存为文件：{filename}")

filename = f"cm2_laplac_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, cm2_laplac)
print(f"数据已保存为文件：{filename}")

filename = f"lw1_laplac_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, lw1_laplac)
print(f"数据已保存为文件：{filename}")

filename = f"lw2_laplac_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, lw2_laplac)
print(f"数据已保存为文件：{filename}")

filename = f"cm1_laplac_std_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, cm1_laplac_std)
print(f"数据已保存为文件：{filename}")

filename = f"cm2_laplac_std_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, cm2_laplac_std)
print(f"数据已保存为文件：{filename}")

filename = f"lw1_laplac_std_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, lw1_laplac_std)
print(f"数据已保存为文件：{filename}")

filename = f"lw2_laplac_std_{description}_{timestamp}.npy"
np.save(SAVDE_DATA_DIR+filename, lw2_laplac_std)
print(f"数据已保存为文件：{filename}")

In [ ]:


# Loading Data 64 voxel
cm1_std_standard = np.load(SAVDE_DATA_DIR+"cm1_laplac_std_SNR(100)_lambda(0.05)_Wmax(500)_US(0)_2025-05-11_22.npy")
cm2_std_standard = np.load(SAVDE_DATA_DIR+"cm2_laplac_std_SNR(100)_lambda(0.05)_Wmax(500)_US(0)_2025-05-11_22.npy")
lw1_std_standard = np.load(SAVDE_DATA_DIR+"lw1_laplac_std_SNR(100)_lambda(0.05)_Wmax(500)_US(0)_2025-05-11_22.npy")
lw2_std_standard = np.load(SAVDE_DATA_DIR+"lw2_laplac_std_SNR(100)_lambda(0.05)_Wmax(500)_US(0)_2025-05-11_22.npy")

cm1_standard_idf = "_SNR(100)_lambda(0.05)_Wmax(500)_US(0)_2025-05-11_22.npy"
cm1_standard = np.load(SAVDE_DATA_DIR+"cm1"+cm1_standard_idf)
cm2_standard = np.load(SAVDE_DATA_DIR+"cm2"+cm1_standard_idf)
cm1_standard_mean = np.mean(cm1_standard,axis=0)
cm2_standard_mean = np.mean(cm2_standard,axis=0)
cm1_std_standard_cg = np.load(SAVDE_DATA_DIR+"cm1_std"+cm1_standard_idf)
cm2_std_standard_cg = np.load(SAVDE_DATA_DIR+"cm2_std"+cm1_standard_idf)

cm1_std_fft = np.load(SAVDE_DATA_DIR+"cm1_fft_std_SNR(100)_lambda(0.05)_Wmax(500)_US(0)_2025-05-14_21.npy")
cm2_std_fft = np.load(SAVDE_DATA_DIR+"cm2_fft_std_SNR(100)_lambda(0.05)_Wmax(500)_US(0)_2025-05-14_21.npy")
lw1_std_fft = np.load(SAVDE_DATA_DIR+"lw1_fft_std_SNR(100)_lambda(0.05)_Wmax(500)_US(0)_2025-05-14_21.npy")
lw2_std_fft = np.load(SAVDE_DATA_DIR+"lw2_fft_std_SNR(100)_lambda(0.05)_Wmax(500)_US(0)_2025-05-14_21.npy")

cm1_fft_idf = "_SNR(100)_lambda(0)_Wmax(500)_US(0)_2025-05-14_19.npy"
cm1_fft = np.load(SAVDE_DATA_DIR+"cm1_fft"+cm1_fft_idf)
cm2_fft = np.load(SAVDE_DATA_DIR+"cm2_fft"+cm1_fft_idf)
cm1_fft_mean = np.mean(cm1_fft,axis=0)
cm2_fft_mean = np.mean(cm2_fft,axis=0)

cm1_std_Zero_Lambda = np.load(SAVDE_DATA_DIR+"cm1_laplac_std_SNR(100)_lambda(0)_Wmax(500)_US(0)_2025-05-11_22.npy")
cm2_std_Zero_Lambda = np.load(SAVDE_DATA_DIR+"cm2_laplac_std_SNR(100)_lambda(0)_Wmax(500)_US(0)_2025-05-11_22.npy")
lw1_std_Zero_Lambda = np.load(SAVDE_DATA_DIR+"lw1_laplac_std_SNR(100)_lambda(0)_Wmax(500)_US(0)_2025-05-11_22.npy")
lw2_std_Zero_Lambda = np.load(SAVDE_DATA_DIR+"lw1_laplac_std_SNR(100)_lambda(0)_Wmax(500)_US(0)_2025-05-11_22.npy")

cm1_Zero_Lambda_idf = "_SNR(100)_lambda(0)_Wmax(500)_US(0)_2025-05-11_22.npy"
cm1_Zero_Lambda = np.load(SAVDE_DATA_DIR+"cm1"+cm1_Zero_Lambda_idf)
cm2_Zero_Lambda = np.load(SAVDE_DATA_DIR+"cm2"+cm1_Zero_Lambda_idf)
cm1_Zero_Lambda_mean = np.mean(cm1_Zero_Lambda,axis=0)
cm2_Zero_Lambda_mean = np.mean(cm2_Zero_Lambda,axis=0)
cm1_std_Zero_Lambda_cg = np.load(SAVDE_DATA_DIR+"cm1_std"+cm1_Zero_Lambda_idf)
cm2_std_Zero_Lambda_cg = np.load(SAVDE_DATA_DIR+"cm2_std"+cm1_Zero_Lambda_idf)


cm1_std_mid_Lambda = np.load(SAVDE_DATA_DIR+"cm1_laplac_std_SNR(100)_lambda(5)_Wmax(500)_US(0)_2025-05-11_22.npy")
cm2_std_mid_Lambda = np.load(SAVDE_DATA_DIR+"cm2_laplac_std_SNR(100)_lambda(5)_Wmax(500)_US(0)_2025-05-11_22.npy")
lw1_std_mid_Lambda = np.load(SAVDE_DATA_DIR+"lw1_laplac_std_SNR(100)_lambda(5)_Wmax(500)_US(0)_2025-05-11_22.npy")
lw2_std_mid_Lambda = np.load(SAVDE_DATA_DIR+"lw2_laplac_std_SNR(100)_lambda(5)_Wmax(500)_US(0)_2025-05-11_22.npy")

cm1_mid_Lambda_idf = "_SNR(100)_lambda(5)_Wmax(500)_US(0)_2025-05-11_22.npy"
cm1_mid_Lambda = np.load(SAVDE_DATA_DIR+"cm1"+cm1_mid_Lambda_idf)
cm2_mid_Lambda = np.load(SAVDE_DATA_DIR+"cm2"+cm1_mid_Lambda_idf)
cm1_mid_Lambda_mean = np.mean(cm1_mid_Lambda,axis=0)
cm2_mid_Lambda_mean = np.mean(cm2_mid_Lambda,axis=0)

cm1_std_high_Lambda = np.load(SAVDE_DATA_DIR+"cm1_laplac_std_SNR(100)_lambda(50)_Wmax(500)_US(0)_2025-05-11_22.npy")
cm2_std_high_Lambda = np.load(SAVDE_DATA_DIR+"cm2_laplac_std_SNR(100)_lambda(50)_Wmax(500)_US(0)_2025-05-11_22.npy")
lw1_std_high_Lambda = np.load(SAVDE_DATA_DIR+"lw1_laplac_std_SNR(100)_lambda(50)_Wmax(500)_US(0)_2025-05-11_22.npy")
lw2_std_high_Lambda = np.load(SAVDE_DATA_DIR+"lw2_laplac_std_SNR(100)_lambda(50)_Wmax(500)_US(0)_2025-05-11_22.npy")

cm1_high_Lambda_idf = "_SNR(100)_lambda(50)_Wmax(500)_US(0)_2025-05-11_22.npy"
cm1_high_Lambda = np.load(SAVDE_DATA_DIR+"cm1"+cm1_high_Lambda_idf)
cm2_high_Lambda = np.load(SAVDE_DATA_DIR+"cm2"+cm1_high_Lambda_idf)
cm1_high_Lambda_mean = np.mean(cm1_high_Lambda,axis=0)
cm2_high_Lambda_mean = np.mean(cm2_high_Lambda,axis=0)

cm1_std_high_Wmax = np.load(SAVDE_DATA_DIR+"cm1_laplac_std_SNR(100)_lambda(0.05)_Wmax(50000)_US(0)_2025-05-11_23.npy")
cm2_std_high_Wmax = np.load(SAVDE_DATA_DIR+"cm2_laplac_std_SNR(100)_lambda(0.05)_Wmax(50000)_US(0)_2025-05-11_23.npy")
lw1_std_high_Wmax = np.load(SAVDE_DATA_DIR+"lw1_laplac_std_SNR(100)_lambda(0.05)_Wmax(50000)_US(0)_2025-05-11_23.npy")
lw2_std_high_Wmax = np.load(SAVDE_DATA_DIR+"lw2_laplac_std_SNR(100)_lambda(0.05)_Wmax(50000)_US(0)_2025-05-11_23.npy")

cm1_high_Wmax_idf = "_SNR(100)_lambda(0.05)_Wmax(50000)_US(0)_2025-05-11_23.npy"
cm1_high_Wmax = np.load(SAVDE_DATA_DIR+"cm1"+cm1_high_Wmax_idf)
cm2_high_Wmax = np.load(SAVDE_DATA_DIR+"cm2"+cm1_high_Wmax_idf)
cm1_high_Wmax_mean = np.mean(cm1_high_Wmax,axis=0)
cm2_high_Wmax_mean = np.mean(cm2_high_Wmax,axis=0)

cm1_std_low_Wmax = np.load(SAVDE_DATA_DIR+"cm1_laplac_std_SNR(100)_lambda(0.05)_Wmax(5)_US(0)_2025-05-11_23.npy")
cm2_std_low_Wmax = np.load(SAVDE_DATA_DIR+"cm2_laplac_std_SNR(100)_lambda(0.05)_Wmax(5)_US(0)_2025-05-11_23.npy")
lw1_std_low_Wmax = np.load(SAVDE_DATA_DIR+"lw1_laplac_std_SNR(100)_lambda(0.05)_Wmax(5)_US(0)_2025-05-11_23.npy")
lw2_std_low_Wmax = np.load(SAVDE_DATA_DIR+"lw2_laplac_std_SNR(100)_lambda(0.05)_Wmax(5)_US(0)_2025-05-11_23.npy")

cm1_low_Wmax_idf = "_SNR(100)_lambda(0.05)_Wmax(5)_US(0)_2025-05-11_23.npy"
cm1_low_Wmax = np.load(SAVDE_DATA_DIR+"cm1"+cm1_low_Wmax_idf)
cm2_low_Wmax = np.load(SAVDE_DATA_DIR+"cm2"+cm1_low_Wmax_idf)
cm1_low_Wmax_mean = np.mean(cm1_low_Wmax,axis=0)
cm2_low_Wmax_mean = np.mean(cm2_low_Wmax,axis=0)

cm1_std_US_50  = np.load(SAVDE_DATA_DIR+"cm1_std_SNR(100)_lambda(0.05)_Wmax(50)_US(50)_2025-05-11_23.npy")
cm2_std_US_50  = np.load(SAVDE_DATA_DIR+"cm2_std_SNR(100)_lambda(0.05)_Wmax(50)_US(50)_2025-05-11_23.npy")
lw1_std_US_50  = np.load(SAVDE_DATA_DIR+"lw1_std_SNR(100)_lambda(0.05)_Wmax(50)_US(50)_2025-05-11_23.npy")
lw2_std_US_50  = np.load(SAVDE_DATA_DIR+"lw2_std_SNR(100)_lambda(0.05)_Wmax(50)_US(50)_2025-05-11_23.npy")

cm1_US_50_idf = "_SNR(100)_lambda(0.05)_Wmax(50)_US(50)_2025-05-11_23.npy"
cm1_US_50 = np.load(SAVDE_DATA_DIR+"cm1"+cm1_US_50_idf)
cm2_US_50 = np.load(SAVDE_DATA_DIR+"cm2"+cm1_US_50_idf)
cm1_US_50_mean = np.mean(cm1_US_50,axis=0)
cm2_US_50_mean = np.mean(cm2_US_50,axis=0)

cm1_std_US_25  = np.load(SAVDE_DATA_DIR+"cm1_std_SNR(100)_lambda(0.05)_Wmax(50)_US(25)_2025-05-12_00.npy")
cm2_std_US_25  = np.load(SAVDE_DATA_DIR+"cm2_std_SNR(100)_lambda(0.05)_Wmax(50)_US(25)_2025-05-12_00.npy")
lw1_std_US_25  = np.load(SAVDE_DATA_DIR+"lw1_std_SNR(100)_lambda(0.05)_Wmax(50)_US(25)_2025-05-12_00.npy")
lw2_std_US_25  = np.load(SAVDE_DATA_DIR+"lw2_std_SNR(100)_lambda(0.05)_Wmax(50)_US(25)_2025-05-12_00.npy")

cm1_US_25_idf = "_SNR(100)_lambda(0.05)_Wmax(50)_US(25)_2025-05-12_00.npy"
cm1_US_25 = np.load(SAVDE_DATA_DIR+"cm1"+cm1_US_25_idf)
cm2_US_25 = np.load(SAVDE_DATA_DIR+"cm2"+cm1_US_25_idf)
cm1_US_25_mean = np.mean(cm1_US_25,axis=0)
cm2_US_25_mean = np.mean(cm2_US_25,axis=0)

cm1_std_full  = np.load(SAVDE_DATA_DIR+"cm1_std_SNR(100)_lambda(0.05)_Wmax(50)_US(0)_2025-05-11_23.npy")
cm2_std_full  = np.load(SAVDE_DATA_DIR+"cm2_std_SNR(100)_lambda(0.05)_Wmax(50)_US(0)_2025-05-11_23.npy")
lw1_std_full  = np.load(SAVDE_DATA_DIR+"lw1_std_SNR(100)_lambda(0.05)_Wmax(50)_US(0)_2025-05-11_23.npy")
lw2_std_full  = np.load(SAVDE_DATA_DIR+"lw2_std_SNR(100)_lambda(0.05)_Wmax(50)_US(0)_2025-05-11_23.npy")

cm1_full_idf = "_SNR(100)_lambda(0.05)_Wmax(50)_US(0)_2025-05-11_23.npy"
cm1_full = np.load(SAVDE_DATA_DIR+"cm1"+cm1_full_idf)
cm2_full = np.load(SAVDE_DATA_DIR+"cm2"+cm1_full_idf)
cm1_full_mean = np.mean(cm1_full,axis=0)
cm2_full_mean = np.mean(cm2_full,axis=0)

cm1_std_fft_US_25 = np.load(SAVDE_DATA_DIR+"cm1_fft_std_SNR(100)_lambda(0.05)_Wmax(500)_US(25)_2025-05-14_21.npy")
cm2_std_fft_US_25 = np.load(SAVDE_DATA_DIR+"cm2_fft_std_SNR(100)_lambda(0.05)_Wmax(500)_US(25)_2025-05-14_21.npy")
lw1_std_fft_US_25 = np.load(SAVDE_DATA_DIR+"lw1_fft_std_SNR(100)_lambda(0.05)_Wmax(500)_US(25)_2025-05-14_21.npy")
lw2_std_fft_US_25 = np.load(SAVDE_DATA_DIR+"lw2_fft_std_SNR(100)_lambda(0.05)_Wmax(500)_US(25)_2025-05-14_21.npy")

cm1_fft_US_25_idf = "_SNR(100)_lambda(0)_Wmax(500)_US(25)_2025-05-14_22.npy"
cm1_fft_US_25 = np.load(SAVDE_DATA_DIR+"cm1_fft"+cm1_fft_US_25_idf)
cm2_fft_US_25 = np.load(SAVDE_DATA_DIR+"cm2_fft"+cm1_fft_US_25_idf)
cm1_fft_US_25_mean = np.mean(cm1_fft_US_25,axis=0)
cm2_fft_US_25_mean = np.mean(cm2_fft_US_25,axis=0)

cm1_std_fft_US_50 = np.load(SAVDE_DATA_DIR+"cm1_fft_std_SNR(100)_lambda(0.05)_Wmax(500)_US(50)_2025-05-14_21.npy")
cm2_std_fft_US_50 = np.load(SAVDE_DATA_DIR+"cm2_fft_std_SNR(100)_lambda(0.05)_Wmax(500)_US(50)_2025-05-14_21.npy")
lw1_std_fft_US_50 = np.load(SAVDE_DATA_DIR+"lw1_fft_std_SNR(100)_lambda(0.05)_Wmax(500)_US(50)_2025-05-14_21.npy")
lw2_std_fft_US_50 = np.load(SAVDE_DATA_DIR+"lw2_fft_std_SNR(100)_lambda(0.05)_Wmax(500)_US(50)_2025-05-14_21.npy")

cm1_fft_US_50_idf = "_SNR(100)_lambda(0.05)_Wmax(500)_US(50)_2025-05-14_21.npy"
cm1_fft_US_50 = np.load(SAVDE_DATA_DIR+"cm1_fft"+cm1_fft_US_50_idf)
cm2_fft_US_50 = np.load(SAVDE_DATA_DIR+"cm2_fft"+cm1_fft_US_50_idf)
cm1_fft_US_50_mean = np.mean(cm1_fft_US_50,axis=0)
cm2_fft_US_50_mean = np.mean(cm2_fft_US_50,axis=0)




In [ ]:
plt.figure(figsize=(9, 5))

# --- Full (0%) ---
sns.lineplot(x=range(len(cm1_std_standard)), y=cm1_std_standard,
             label="Glu (Full)", color='tab:blue', linewidth=2, alpha=1.0)
# sns.lineplot(x=range(len(cm2_std_standard)), y=cm2_std_standard,
#              label="Cho (Full)", color='tab:orange', linewidth=2, alpha=1.0)

sns.lineplot(x=range(len(cm1_std_fft)), y=cm1_std_fft,
             label="Glu (Full)", color='tab:red', linewidth=2, alpha=1,linestyle = '-')

# --- Undersample 25% ---
sns.lineplot(x=range(len(cm1_std_US_25)), y=cm1_std_US_25,
             label="Glu (US 25%)", color='tab:blue', linewidth=2, alpha=0.7)
# sns.lineplot(x=range(len(cm2_std_US_25)), y=cm2_std_US_25,
#              label="Cho (US 25%)", color='tab:orange', linewidth=2, alpha=0.7)

# --- Undersample 25% ---
sns.lineplot(x=range(len(cm1_std_fft_US_25)), y=cm1_std_fft_US_25,
             label="Glu (US 25%)", color='tab:red', linewidth=2, alpha=0.7,linestyle = '-')
# sns.lineplot(x=range(len(cm2_std_fft_US_25)), y=cm2_std_fft_US_25,
#              label="Cho (US 25%)", color='tab:orange', linewidth=2, alpha=0.7,linestyle = '--')

# --- Undersample 50% ---
sns.lineplot(x=range(len(cm1_std_US_50)), y=cm1_std_US_50,
             label="Glu (US 50%)", color='tab:blue', linewidth=2, alpha=0.3)
# sns.lineplot(x=range(len(cm2_std_US_50)), y=cm2_std_US_50,
#              label="Cho (US 50%)", color='tab:orange', linewidth=2, alpha=0.3)

# --- Undersample 50% ---
sns.lineplot(x=range(len(cm1_std_fft_US_50)), y=cm1_std_fft_US_50,
             label="Glu (US 50%)", color='tab:red', linewidth=2, alpha=0.3,linestyle = '-')
# sns.lineplot(x=range(len(cm2_std_fft_US_50)), y=cm2_std_fft_US_50,
#              label="Cho (US 50%)", color='tab:orange', linewidth=2, alpha=0.3,linestyle = '--')



# --- Reference ---
sns.lineplot(x=range(len(water_rou_1D)), y=water_rou_1D * 0.06,
             label='Anatomical Reference', linestyle=':', color='black', linewidth=2)

plt.xlabel("Voxel Index", fontweight='bold')
plt.ylabel("Uncertainty", fontweight='bold')
plt.title("Concentration Uncertainty", fontweight='bold')
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), prop={'weight': 'bold'})
plt.tight_layout()
plt.show()



plt.figure(figsize=(9, 5))
# plt.figure(figsize=(6, 5))

# --- FFT 25 ---
sns.lineplot(x=np.arange(len(cm1_fft_mean))+1, y=cm1_fft_mean,
              color='tab:red', linewidth=2, alpha=1)
sns.lineplot(x=range(len(cm1_fft_US_25_mean)), y=cm1_fft_US_25_mean,
              color='tab:red', linewidth=2, alpha=0.7)
sns.lineplot(x=range(len(cm1_fft_US_25_mean)), y=cm1_fft_US_50_mean,
              color='tab:red', linewidth=2, alpha=0.3)

sns.lineplot(x=range(len(cm1_standard_mean)), y=cm1_standard_mean,
              color='tab:blue', linewidth=2, alpha=1.0)

# --- SPICE 25 ---
sns.lineplot(x=range(len(cm1_US_25_mean)), y=cm1_US_25_mean,
              color='tab:blue', linewidth=2, alpha=0.7)
sns.lineplot(x=range(len(cm1_US_25_mean)), y=cm1_US_50_mean,
              color='tab:blue', linewidth=2, alpha=0.3)
# sns.lineplot(x=range(len(cm2_US_25_mean)), y=cm2_US_25_mean,
#               color='tab:orange', linewidth=2, alpha=1)


# sns.lineplot(x=range(len(cm2_fft_US_25_mean)), y=cm2_fft_US_25_mean,
#               color='tab:orange', linewidth=2, alpha=0.3)



# # --- Reference ---
sns.lineplot(x=range(len(C0_GT)), y=C0_GT,
              color='black', linewidth=2, alpha=1, linestyle=':')
# sns.lineplot(x=range(len(C1_GT)), y=C1_GT,
#               color='magenta', linewidth=2, alpha=0.2, linestyle='--')

# plt.xlim(35, 42)  # Zoom into voxel index range 
# plt.ylim(0.4, 0.8)  # Zoom into voxel index range
plt.xlabel("Voxel Index", fontweight="bold")
plt.ylabel("Concentration", fontweight="bold")
plt.title("Concentration mean", fontweight="bold")
# plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), prop={'weight': 'bold'})
plt.tight_layout()
plt.show()


In [ ]:

# Set seaborn style to white
sns.set(style="whitegrid")

# === Plot Cm uncertainty ===
plt.figure(figsize=(9, 5))

# Zero λ
sns.lineplot(x=range(len(cm1_std_Zero_Lambda)), y=cm1_std_Zero_Lambda,
             label="No Constraint(λ = 0)", linestyle='--', color='tab:blue', linewidth=2, alpha=0.3)
# sns.lineplot(x=range(len(cm2_std_Zero_Lambda)), y=cm2_std_Zero_Lambda,
#              label="Cho No Constraint", linestyle='-', color='tab:orange', linewidth=2, alpha=1.0)

# Standard λ
sns.lineplot(x=range(len(cm1_std_standard)), y=cm1_std_standard,
             label="Moderate λ", linestyle='-', color='tab:blue', linewidth=2, alpha=0.7)
# sns.lineplot(x=range(len(cm2_std_standard)), y=cm2_std_standard,
#              label="Cho Moderate λ", linestyle='-', color='tab:orange', linewidth=2, alpha=0.7)

# High λ
sns.lineplot(x=range(len(cm1_std_mid_Lambda)), y=cm1_std_mid_Lambda,
             label="High λ", linestyle='-', color='tab:blue', linewidth=2, alpha=1)
# sns.lineplot(x=range(len(cm2_std_mid_Lambda)), y=cm2_std_mid_Lambda,
#              label="Cho High λ", linestyle='-', color='tab:orange', linewidth=2, alpha=0.3)



# Reference
sns.lineplot(x=range(len(water_rou_1D)), y=water_rou_1D * 0.04,
             label='Anatomical Reference', linestyle=':', color='gray', linewidth=2)

plt.xlabel("Voxel Index", fontweight='bold')
plt.ylabel("Uncertainty", fontweight='bold')
plt.title("Concentration Uncertainty (λ)", fontweight='bold')
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), prop={'weight': 'bold'})
plt.tight_layout()
plt.show()

plt.figure(figsize=(9, 5))
# plt.figure(figsize=(6, 5))
# --- Reference ---
sns.lineplot(x=range(len(C0_GT)), y=C0_GT,
              label='Ground Truth',color='black', linewidth=2, alpha=1, linestyle=':')

# Zero λ
sns.lineplot(x=range(len(cm1_Zero_Lambda_mean)), y=cm1_Zero_Lambda_mean,
             label="No Constraint(λ = 0)", linestyle='--', color='tab:blue', linewidth=3, alpha=0.3)
# sns.lineplot(x=range(len(cm2_std_Zero_Lambda)), y=cm2_std_Zero_Lambda,
#              label="Cho No Constraint", linestyle='-', color='tab:orange', linewidth=2, alpha=1.0)

# Standard λ
sns.lineplot(x=range(len(cm1_standard_mean)), y=cm1_standard_mean,
             label="Moderate λ", linestyle='-', color='tab:blue', linewidth=2, alpha=0.7)
# sns.lineplot(x=range(len(cm2_std_standard)), y=cm2_std_standard,
#              label="Cho Moderate λ", linestyle='-', color='tab:orange', linewidth=2, alpha=0.7)

# High λ
sns.lineplot(x=range(len(cm1_mid_Lambda_mean)), y=cm1_mid_Lambda_mean,
             label="High λ", linestyle='-', color='tab:blue', linewidth=2, alpha=1)
# sns.lineplot(x=range(len(cm2_std_mid_Lambda)), y=cm2_std_mid_Lambda,
#              label="Cho High λ", linestyle='-', color='tab:orange', linewidth=2, alpha=0.3)




# sns.lineplot(x=range(len(C1_GT)), y=C1_GT,
#               color='magenta', linewidth=2, alpha=0.2, linestyle='--')

plt.xlabel("Voxel Index", fontweight='bold')
plt.ylabel("concentration", fontweight='bold')
plt.title("Concentration Mean (λ)", fontweight='bold')
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), prop={'weight': 'bold'})
plt.tight_layout()
plt.show()


plt.figure(figsize=(6, 5))
# --- Reference ---
sns.lineplot(x=range(len(C0_GT)), y=C0_GT,
              color='black', linewidth=2, alpha=1, linestyle=':')

# Zero λ
sns.lineplot(x=range(len(cm1_Zero_Lambda_mean)), y=cm1_Zero_Lambda_mean,
             linestyle='--', color='tab:blue', linewidth=2, alpha=0.3)
# sns.lineplot(x=range(len(cm2_std_Zero_Lambda)), y=cm2_std_Zero_Lambda,
#              label="Cho No Constraint", linestyle='-', color='tab:orange', linewidth=2, alpha=1.0)

# Standard λ
sns.lineplot(x=range(len(cm1_standard_mean)), y=cm1_standard_mean,
             linestyle='-', color='tab:blue', linewidth=2, alpha=0.7)
# sns.lineplot(x=range(len(cm2_std_standard)), y=cm2_std_standard,
#              label="Cho Moderate λ", linestyle='-', color='tab:orange', linewidth=2, alpha=0.7)

# High λ
sns.lineplot(x=range(len(cm1_mid_Lambda_mean)), y=cm1_mid_Lambda_mean,
              linestyle='-', color='tab:blue', linewidth=2, alpha=1)
# sns.lineplot(x=range(len(cm2_std_mid_Lambda)), y=cm2_std_mid_Lambda,
#              label="Cho High λ", linestyle='-', color='tab:orange', linewidth=2, alpha=0.3)




# sns.lineplot(x=range(len(C1_GT)), y=C1_GT,
#               color='magenta', linewidth=2, alpha=0.2, linestyle='--')

plt.xlim(30, 40)  # Zoom into voxel index range 
plt.ylim(0.15, 0.6)  # Zoom into voxel index range
# plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), prop={'weight': 'bold'})
plt.tight_layout()
plt.show()








In [ ]:

# === Plot Cm uncertainty (Dw) ===
plt.figure(figsize=(9, 5))

# High
sns.lineplot(x=range(len(cm1_std_high_Wmax)), y=cm1_std_high_Wmax,
             label="High Dw", linestyle='-', color='tab:blue', linewidth=2, alpha=1)
# sns.lineplot(x=range(len(cm2_std_high_Wmax)), y=cm2_std_high_Wmax,
#              label="Cho High Dw", linestyle='-', color='tab:orange', linewidth=2, alpha=0.3)

# Low
sns.lineplot(x=range(len(cm1_std_low_Wmax)), y=cm1_std_low_Wmax,
             label="Low Dw", linestyle='-', color='tab:blue', linewidth=2, alpha=0.3)
# sns.lineplot(x=range(len(cm2_std_low_Wmax)), y=cm2_std_low_Wmax,
#              label="Cho Low Dw", linestyle='-', color='tab:orange', linewidth=2, alpha=1.0)

# # Standard
# sns.lineplot(x=range(len(cm1_std_standard)), y=cm1_std_standard,
#              label="NAA Moderate Dw", linestyle='-', color='tab:blue', linewidth=2, alpha=0.7)
# sns.lineplot(x=range(len(cm2_std_standard)), y=cm2_std_standard,
#              label="Cho Moderate Dw", linestyle='-', color='tab:orange', linewidth=2, alpha=0.7)


# --- Reference ---
sns.lineplot(x=range(len(water_rou_1D)), y=water_rou_1D * 0.04,
             label='Anatomical Reference', linestyle=':', color='gray', linewidth=2)

plt.xlabel("Voxel Index", fontweight='bold')
plt.ylabel("Uncertainty", fontweight='bold')
plt.title("Concentration Uncertainty (Dw)", fontweight='bold')
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), prop={'weight': 'bold'})
plt.tight_layout()
plt.show()

plt.figure(figsize=(9, 5))
# plt.figure(figsize=(6, 5))

# --- Reference ---
sns.lineplot(x=range(len(C0_GT)), y=C0_GT,
              label = 'Ground Truth',color='black', linewidth=2, alpha=1, linestyle='--')

# --- High Dw ---
sns.lineplot(x=range(len(cm1_high_Wmax_mean)), y=cm1_high_Wmax_mean,
              label = 'High Dw',color='tab:blue', linewidth=2, alpha=1.0)
# sns.lineplot(x=range(len(cm2_high_Wmax_mean)), y=cm2_high_Wmax_mean,
#               color='tab:orange', linewidth=2, alpha=1.0)

# --- Low Dw ---
sns.lineplot(x=range(len(cm1_low_Wmax_mean)), y=cm1_low_Wmax_mean,
              label = 'Low Dw',color='tab:blue', linewidth=2, alpha=0.4)
# sns.lineplot(x=range(len(cm2_low_Wmax_mean)), y=cm2_low_Wmax_mean,
#               color='tab:orange', linewidth=2, alpha=0.4)



# sns.lineplot(x=range(len(C1_GT)), y=C1_GT,
#               color='magenta', linewidth=2, alpha=0.2, linestyle='--')

# plt.xlim(35, 42)  # Zoom into voxel index range 
# plt.ylim(0.4, 0.8)  # Zoom into voxel index range
plt.xlabel("Voxel Index", fontweight='bold')
plt.ylabel("concentration", fontweight='bold')
plt.title("Concentration Mean (Dw)", fontweight='bold')
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), prop={'weight': 'bold'})
plt.tight_layout()
plt.show()

plt.figure(figsize=(6, 5))

# --- Reference ---
sns.lineplot(x=range(len(C0_GT)), y=C0_GT,
              color='black', linewidth=2, alpha=1, linestyle='--')

# --- High Dw ---
sns.lineplot(x=range(len(cm1_high_Wmax_mean)), y=cm1_high_Wmax_mean,
              color='tab:blue', linewidth=2, alpha=1.0)
# sns.lineplot(x=range(len(cm2_high_Wmax_mean)), y=cm2_high_Wmax_mean,
#               color='tab:orange', linewidth=2, alpha=1.0)

# --- Low Dw ---
sns.lineplot(x=range(len(cm1_low_Wmax_mean)), y=cm1_low_Wmax_mean,
              color='tab:blue', linewidth=2, alpha=0.4)
# sns.lineplot(x=range(len(cm2_low_Wmax_mean)), y=cm2_low_Wmax_mean,
#               color='tab:orange', linewidth=2, alpha=0.4)



# sns.lineplot(x=range(len(C1_GT)), y=C1_GT,
#               color='magenta', linewidth=2, alpha=0.2, linestyle='--')

plt.xlim(35, 42)  # Zoom into voxel index range 
plt.ylim(0.4, 0.8)  # Zoom into voxel index range
# plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), prop={'weight': 'bold'})
plt.tight_layout()
plt.show()




In [ ]:
plt.figure(figsize=(9, 5))

# --- FFT ---
sns.lineplot(x=range(len(cm1_std_fft)), y=cm1_std_fft,
             label="Glu (FFT)", color='tab:red', linewidth=2, alpha=1)
sns.lineplot(x=range(len(cm2_std_fft)), y=cm2_std_fft,
             label="Cho (FFT)", color='tab:orange', linewidth=2, alpha=0.5)


# --- without constraint ---
sns.lineplot(x=range(len(cm1_std_Zero_Lambda_cg)), y=cm1_std_Zero_Lambda_cg,
             label="Glu (SPICE)", color='tab:blue', linewidth=3, alpha=0.5, linestyle='--')
sns.lineplot(x=range(len(cm2_std_Zero_Lambda_cg)), y=cm2_std_Zero_Lambda_cg,
             label="Cho (SPICE)", color='tab:cyan', linewidth=3, alpha=0.5, linestyle='--')



# --- with constraint ---
sns.lineplot(x=range(len(cm1_std_standard_cg)), y=cm1_std_standard_cg,
             label="Glu (SPICE + constraint)", color='tab:blue', linewidth=2, alpha=1)
sns.lineplot(x=range(len(cm2_std_standard_cg)), y=cm2_std_standard_cg,
             label="Cho (SPICE + constraint)", color='tab:cyan', linewidth=2, alpha=1)



# --- Reference ---
sns.lineplot(x=range(len(water_rou_1D)), y=water_rou_1D * 0.03,
             label='Anatomical Reference', linestyle=':', color='gray', linewidth=2)

plt.xlabel("Voxel Index", fontweight='bold')
plt.ylabel("Uncertainty", fontweight='bold')
plt.title("Concentration Uncertainty after Fitting", fontweight='bold')
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), prop={'weight': 'bold'})
plt.tight_layout()
plt.show()

